In [ ]:
#@title Libraries & Functions (Run this cell First)
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as iw
from IPython.display import clear_output,display
import scipy.optimize as sco
import scipy.integrate as sci
import sympy as sp
pd.set_option('display.max_columns', None)

color_v=["red","blue","orange","gray","magenta","green","cyan","purple","olive","pink","brown","black","yellow"]  #Assign color to each bar
marker=["o","v","d","$*$","x","^","P",(5,2),(5,1),"H","s","4",(8,1),8,9]  #Symbols to lines
label_exc=["pH","P(bar)","yCO2","yH2","yCO","H2(mmol)","CO2(mmol)","qH2(mmol/gcell/h)","But:Hex(mol/mol)","qBut(g/gcell/h)","qHex(g/gcell/h)"]   #Label exceptions for adding "(g/L)" on y-axis plot
linestyle=[":","--","-."]
#color_v=["red","gray","blue","magenta","green","orange","cyan","purple","olive","pink"]  #Assign color to each bar
#marker=["o","v","d","$*$","x","^","P",(5,2),(5,1)]  #Symbols to lines

from mpl_toolkits.mplot3d import Axes3D
import plotly
import plotly.graph_objects as go
from matplotlib import cm

import scipy.stats as scs

def umax_calc(df,nc):
  umax_vec=pd.DataFrame()
  for j in range(1,nc+1):
    X_exp,time=df.values[:,j],df.values[:,0]
    logN=np.log(X_exp/X_exp[0])
    slope0=(logN[1:]-logN[:-1])/(time[1:]-time[:-1])
    ind_sl0=np.where(slope0>=max(slope0)*0.8)[0]
    umax0=scs.linregress(x=time[ind_sl0+1],y=logN[ind_sl0+1],).slope if len(ind_sl0)>1 else slope0[ind_sl0[0]]
    umax_vec[list(df.columns)[j]]=[umax0]
  return(umax_vec)

def yields(P,Pstd,Sust):
  if type(Sust)==np.ndarray:
    sust=Sust
  else:
    sust=sum(Sust)
  v_yield=(P-P[0])/(sust[0]-sust)
  v_yield[0]=0
  v_yield[v_yield<0]=0
  v_yieldstd=(Pstd-Pstd[0])/(sust[0]-sust)
  v_yieldstd[0]=0
  v_yieldstd[v_yieldstd<0]=0
  return v_yield,v_yieldstd

def plotyield(v_yield,vyieldstd,ylabel):
  for i in range(len(v_yield[0])):
    plt.errorbar(dfmeans.values[:,0],v_yield[:,i],yerr=vyieldstd[:,i],capsize=3,label=series_list[i])
  plt.xlabel(list(df.columns)[0])
  plt.ylabel(ylabel)
  plt.legend()
  plt.show()

def show_inline_matplotlib_plots():
    """Show matplotlib plots immediately if using the inline backend.

    With ipywidgets 6.0, matplotlib plots don't work well with interact when
    using the inline backend that comes with ipykernel. Basically, the inline
    backend only shows the plot after the entire cell executes, which does not
    play well with drawing plots inside of an interact function. See
    https://github.com/jupyter-widgets/ipywidgets/issues/1181/ and
    https://github.com/ipython/ipython/issues/10376 for more details. This
    function displays any matplotlib plots if the backend is the inline backend.
    """
    try:
        import matplotlib as mpl
        from ipykernel.pylab.backend_inline import flush_figures
    except ImportError:
        return

    if mpl.get_backend() == 'module://ipykernel.pylab.backend_inline':
        flush_figures()

def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

def plotfcont(compx,xlabel,ylabel,data):
  global full_data
  z_vec=search_comps(df_list[0],compx).values[0]
  for i in range(1,len(df_list)):
    z_vec=np.append(z_vec,search_comps(df_list[i],compx).values[0],axis=None)
  full_data=np.c_[data,z_vec]
  x = data[:, 0]
  y = data[:, 1]
  z = z_vec
  lista=[]
  figf1=plt.figure()
  plt.tricontourf(x,y,z,levels=10,alpha=0.7)
  plt.colorbar(label=str(compx)+" Conc. [g/L]") if compx not in label_exc else  plt.colorbar(label=str(compx))
  plt.scatter(x,y,c="red")
  for i in range(len(z)):
    plt.annotate(round(z[i],3),(x[i], y[i]))
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  #plt.show()
  return figf1

def plot3d(compx,xlabel,ylabel,data):
  z_vec=search_comps(df_list[0],compx).values[0]
  for i in range(1,len(df_list)):
    z_vec=np.append(z_vec,search_comps(df_list[i],compx).values[0],axis=None)
  full_data=np.c_[data,z_vec]
  x = data[:, 0]
  y = data[:, 1]
  z = z_vec
  lista=[]
  #lista.append(go.Mesh3d(x=data[:, 0], y=data[:, 1], z=z_vec, opacity=0.8,))
  lista.append(go.Mesh3d(x=data[:, 0], y=data[:, 1], z=z_vec, opacity=0.8,facecolor=['rgb('+str((z_i-min(z_vec))/min(z_vec)*30)+',100,'+str(100-(z_i-min(z_vec))/min(z_vec)*30)+')' for z_i in z_vec]))
  z_legend= search_comps(df_list[0],compx).columns[0].split(" ")[-1]
  z_legend=z_legend + " (g/L)" if z_legend not in label_exc else z_legend
  layouts=go.Layout(margin=dict(r=10, l=10, b=10, t=10),scene=dict(xaxis=dict(title=xlabel),yaxis=dict(title=ylabel),zaxis=dict(title=z_legend)))
  fig = go.Figure(data=lista, layout=layouts)
  fig.show()

def search_comps(df,comp):
  index=[comp in coli for coli in df.columns]
  vec=df[df.columns[index]]
  return vec

def Plotbar_series(comp_list,comp_list2,series_list,times_list):
  Pseries=len(comp_list)+len(comp_list2)
  #barw=(min(dfmeans.iloc[1:,0].values-dfmeans.iloc[:-1,0].values))/Pseries*0.8
  ind_max1=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
  ind_max2=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
  ind_time=dfmeans.loc[dfmeans[dfmeans.columns[0]].isin(times_list)].index
  barw=(min(dfmeans.iloc[ind_time,0].values[1:]-dfmeans.iloc[ind_time,0].values[:-1]))/Pseries*0.8
  ymax=(np.max(dfmeans.iloc[ind_time,ind_max1].values)+np.max(dfstd.iloc[:,ind_max1].values))*1.1
  try:
    y2max=(np.max(dfmeans.iloc[ind_time,ind_max2].values)+np.max(dfstd.iloc[:,ind_max2].values))*1.1
  except:
    None
  ylabel1=yLabels(comp_list,label_exc)
  ylabel2=yLabels(comp_list2,label_exc)

  for j in range(len(series_list)):
    index=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and (any([comp_list[i] in x[0] for i in range(len(comp_list))]) or any([comp_list2[i] in x[0] for i in range(len(comp_list2))])),axis=1).values
    ind_ax1=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
    ind_ax2=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
    fig,ax = plt.subplots()
    dfmeans2=dfmeans.iloc[ind_time,index]
    dfstd2=dfstd.iloc[ind_time,index]
    if len(comp_list2)>0:
      ax2=ax.twinx()
    for i in np.arange(len(dfmeans2.columns)):
      if any(comp_list[h] in dfmeans2.columns[i] for h in range(len(comp_list))):
        ax.bar(x=dfmeans.iloc[ind_time,0]+i*barw,
              height=dfmeans2.iloc[:,i],yerr=dfstd2.iloc[:,i],
              align="center",width=barw,
              label=dfmeans2.columns[i].split(" ")[-1],capsize=4,color=color_v[i])
      else:
        ax2.bar(x=dfmeans.iloc[ind_time,0]+i*barw,
              height=dfmeans2.iloc[:,i],yerr=dfstd2.iloc[:,i],
              align="center",width=barw,
              label=dfmeans2.columns[i].split(" ")[-1],capsize=4,color=color_v[i])
    try:
      ax.set_ylim(0,ymax)
    except:
      ax.set_ylim(0)
    ax.legend(loc=(1.15,0.5))
    ax.set_ylabel(ylabel1)
    try:
      ax2.set_ylabel(ylabel2)
      ax2.legend(loc=(1.15,0.0))
      try:
        ax2.set_ylim(0,y2max)
      except:
        ax2.set_ylim(0)
    except:
      None
    ax.set_xlabel(dfmeans.columns[0])
    ax.set_xticks(dfmeans.iloc[ind_time,0]+barw*(Pseries-1)/2)
    ax.set_xticklabels(np.int_(dfmeans.iloc[ind_time,0]))
    xlim1,xlim2=ax.get_xlim()
    xtick_width=barw*Pseries/(max(dfmeans.iloc[ind_time,0].values)-barw*Pseries/2)*(xlim2-xlim1)
    ax.xaxis.set_tick_params(width=0.1,length=0)
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    axwidth,axheight = bbox.width, bbox.height

def Plotline_series(comp_list,comp_list2,series_list,times_list):
  Pseries=len(comp_list)
  ind_max1=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
  ind_max2=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
  ind_time=dfmeans.loc[dfmeans[dfmeans.columns[0]].isin(times_list)].index
  ymax=(np.max(dfmeans.iloc[ind_time,ind_max1].values)+np.max(dfstd.iloc[:,ind_max1].values))*1.1
  try:
    y2max=(np.max(dfmeans.iloc[ind_time,ind_max2].values)+np.max(dfstd.iloc[:,ind_max2].values))*1.1
  except:
    None
  ylabel1=yLabels(comp_list,label_exc)
  ylabel2=yLabels(comp_list2,label_exc)
  for j in range(len(series_list)):
    index=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and (any([comp_list[i] in x[0] for i in range(len(comp_list))]) or any([comp_list2[i] in x[0] for i in range(len(comp_list2))])),axis=1).values
    ind_ax1=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
    ind_ax2=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
    fig,ax = plt.subplots()
    dfmeans2=dfmeans.iloc[ind_time,index]
    dfstd2=dfstd.iloc[ind_time,index]
    if len(comp_list2)>0:
      ax2=ax.twinx()
    for i in np.arange(len(dfmeans2.columns)):
      if any(comp_list[h] in dfmeans2.columns[i] for h in range(len(comp_list))):
        ax.errorbar(x=dfmeans.iloc[ind_time,0],y=dfmeans2.iloc[:,i],
                    yerr=dfstd2.iloc[:,i],fmt=':',color=color_v[i],
                    marker=marker[i],markersize=8,label=dfmeans2.columns[i].split(" ")[-1],
                    capsize=4,elinewidth=1)
      else:
        ax2.errorbar(x=dfmeans.iloc[ind_time,0],y=dfmeans2.iloc[:,i],
                    yerr=dfstd2.iloc[:,i],fmt=':',color=color_v[i],
                    marker=marker[i],markersize=8,label=dfmeans2.columns[i].split(" ")[-1],
                    capsize=4,elinewidth=1)
    try:
      ax.set_ylim(0,ymax)
    except:
      ax.set_ylim(0)
    ax.legend(loc=(1.15,0.5))
    ax.set_ylabel(ylabel1)
    try:
      ax2.set_ylabel(ylabel2)
      ax2.legend(loc=(1.15,0.0))
      try:
        ax2.set_ylim(0,y2max)
      except:
        ax2.set_ylim(0)
    except:
      None
    ax.set_xlabel(dfmeans.columns[0])
    ax.set_xticks(dfmeans.iloc[ind_time,0])
    ax.set_xlim(0)

def Plotbar_comps(comp_list,comp_list2,series_list,times_list):
  Pseries=len(series_list)
  ind_time=dfmeans.loc[dfmeans[dfmeans.columns[0]].isin(times_list)].index
  barw=(min(dfmeans.iloc[ind_time,0].values[1:]-dfmeans.iloc[ind_time,0].values[:-1]))/Pseries*0.8
  for j in range(len(comp_list)):
    ylabel1=yLabels([comp_list[j],],label_exc)
    index=pd.DataFrame(dfmeans.columns).apply(lambda x: comp_list[j] in x[0] and any([series_list[i] in x[0] for i in range(len(series_list))]),axis=1).values
    fig,ax = plt.subplots()
    dfmeans2=dfmeans.iloc[ind_time,index]
    dfstd2=dfstd.iloc[ind_time,index]
    for i in np.arange(len(dfmeans2.columns)):
      ax.bar(x=dfmeans.iloc[ind_time,0]+i*barw,
             height=dfmeans2.iloc[:,i],yerr=dfstd2.iloc[:,i],
             align="center",width=barw,
             label=dfmeans2.columns[i].split(" ")[0],capsize=4,color=color_v[i])
    ax.set_ylabel(str(comp_list[j])+" Conc. [g/L]") if comp_list[j] not in label_exc else ax.set_ylabel(comp_list[j])
    ax.set_ylabel(ylabel1)
    ax.set_ylim(0)
    ax.legend(loc=(1.1,0))
    ax.set_xlabel(dfmeans.columns[0])
    ax.set_xticks(dfmeans.iloc[ind_time,0]+barw*(Pseries-1)/2)
    ax.set_xticklabels(np.int_(dfmeans.iloc[ind_time,0]))
    xlim1,xlim2=ax.get_xlim()
    xtick_width=barw*Pseries/(max(dfmeans.iloc[ind_time,0].values)-barw*Pseries/2)*(xlim2-xlim1)
    ax.xaxis.set_tick_params(width=0.1,length=0,pad=10)


def Plotline_comps(comp_list,comp_list2,series_list,times_list):
  Pseries=len(series_list)
  ind_time=dfmeans.loc[dfmeans[dfmeans.columns[0]].isin(times_list)].index
  for j in range(len(comp_list)):
    ylabel1=yLabels([comp_list[j],],label_exc)
    index=pd.DataFrame(dfmeans.columns).apply(lambda x: comp_list[j] in x[0] and any([series_list[i] in x[0] for i in range(len(series_list))]),axis=1).values
    fig,ax = plt.subplots()
    dfmeans2=dfmeans.iloc[ind_time,index]
    dfstd2=dfstd.iloc[ind_time,index]
    for i in np.arange(len(dfmeans2.columns)):
      ax.errorbar(x=dfmeans.iloc[ind_time,0],y=dfmeans2.iloc[:,i],
                  yerr=dfstd2.iloc[:,i],fmt=':',color=color_v[i],
                  marker=marker[j],markersize=8,label=dfmeans2.columns[i].split(" ")[0],
                  capsize=4,elinewidth=1)
    ax.set_ylabel(ylabel1)
    try:
      ax.set_ylim(0,ymax)   #If ymax is declared to compare with other activities
    except:
      ax.set_ylim(0)
    ax.set_xlabel(dfmeans.columns[0])
    ax.set_xticks(dfmeans.iloc[ind_time,0])
    ax.legend(loc=(1.1,0))
    ax.set_xlim(0)

def Plotline_all(comp_list,comp_list2,series_list,times_list):
  Pseries=len(comp_list)
  ind_max1=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
  ind_max2=pd.DataFrame(dfmeans.columns).apply(lambda x: any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
  ind_time=dfmeans.loc[dfmeans[dfmeans.columns[0]].isin(times_list)].index
  ypmax=(np.max(dfmeans.iloc[ind_time,ind_max1].values)+np.max(dfstd.iloc[:,ind_max1].values))*1.1 if "ymax" not in globals() else ymax
  try:
    y2max=(np.max(dfmeans.iloc[ind_time,ind_max2].values)+np.max(dfstd.iloc[:,ind_max2].values))*1.1
  except:
    None
  ylabel1=yLabels(comp_list,label_exc)
  ylabel2=yLabels(comp_list2,label_exc)
  fig,ax = plt.subplots()
  if len(comp_list2)>0:
    ax2=ax.twinx()
  for j in range(len(series_list)):
    index=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and (any([comp_list[i] in x[0] for i in range(len(comp_list))]) or any([comp_list2[i] in x[0] for i in range(len(comp_list2))])),axis=1).values
    ind_ax1=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list[i] in x[0] for i in range(len(comp_list))]),axis=1).values
    ind_ax2=pd.DataFrame(dfmeans.columns).apply(lambda x: series_list[j] in x[0] and any([comp_list2[i] in x[0] for i in range(len(comp_list2))]),axis=1).values
    dfmeans2=dfmeans.iloc[ind_time,index]
    dfstd2=dfstd.iloc[ind_time,index]
    for i in np.arange(len(dfmeans2.columns)):
      if any(comp_list[h] in dfmeans2.columns[i] for h in range(len(comp_list))):
        ax.errorbar(x=dfmeans.iloc[ind_time,0],y=dfmeans2.iloc[:,i],
                    yerr=dfstd2.iloc[:,i],fmt=linestyle[i],color=color_v[j],
                    marker=marker[i],markersize=8,label=dfmeans2.columns[i],
                    capsize=4,elinewidth=1)
      else:
        ax2.errorbar(x=dfmeans.iloc[ind_time,0],y=dfmeans2.iloc[:,i],
                    yerr=dfstd2.iloc[:,i],fmt=linestyle[i],color=color_v[j],
                    marker=marker[i],markersize=8,label=dfmeans2.columns[i],
                    capsize=4,elinewidth=1)
    ax.set_ylim(0,ypmax)
    ax.legend(loc=(1.15,0.5))
    ax.set_ylabel(ylabel1)
    try:
      ax2.set_ylabel(ylabel2)
      ax2.legend(loc=(1.15,0.0))
      try:
        ax2.set_ylim(0,y2max)
      except:
        ax2.set_ylim(0)
    except:
      None
    ax.set_xlabel(dfmeans.columns[0])
    ax.set_xticks(dfmeans.iloc[ind_time,0])
    ax.set_xlim(0)
#############################################



def yLabels(comp_list,label_exc):
  try:
    ylabel,exc_label="",[]
    for x in comp_list:
      if x not in label_exc:
        ylabel+=x+", "
      else:
        exc_label.append(x)
    if len(ylabel)>0:
      ylabel=ylabel[:-2]+" [g/L], " if len(exc_label)>0 else ylabel[:-2]+" [g/L]"
    if len(exc_label)>0:
      for x in exc_label:
        ylabel+=x+", "
      ylabel=ylabel[:-2]
    return ylabel
  except:
    None

def DF_build(df1,Nr,IDc,Choice,replace_series=False):
  global dfmeans,dfstd,series_list,comp_list
  dfmeans=df.apply(lambda x: [x[0]]+[np.mean(x[i:i+Nr]) for i in range(1,len(x),Nr)],axis=1,result_type="expand")
  try:
    dfmeans.columns=np.array([df.columns[0]]+[str(replace_series[(i-1)%(Nr*len(replace_series))//Nr]+" "+df.columns[i][IDc+1:]) for i in range(1,len(df.columns),Nr)])
  except:
    dfmeans.columns=np.array([df.columns[0]]+[str(df.columns[i][:IDc]+" "+df.columns[i][IDc+1:]) for i in range(1,len(df.columns),Nr)])
  dfstd=df.apply(lambda x: [x[0]]+[np.std(x[i:i+Nr]) for i in range(1,len(x),Nr)],axis=1,result_type="expand")
  dfstd.columns=dfmeans.columns
  sheet_comps=[col.split()[1] for col in dfmeans.columns[1:]]   #After split goes the word with the compound and in columns, except the "time" column
  comp_ind=np.unique(sheet_comps,return_index=True)[1]
  comp_list=[sheet_comps[index] for index in sorted(comp_ind)]
  sheet_series=[col.split()[0] for col in dfmeans.columns[1:]]   #Before split goes the word with the series name and in columns, except the "time" column
  series_ind=np.unique(sheet_series,return_index=True)[1]
  series_list=[sheet_series[index] for index in sorted(series_ind)]
  if Choice!=None:
    Select_comps=iw.SelectMultiple(options=comp_list,value=comp_list[1:],rows=10,description='Y-axis:',disabled=False)
    Select_comps2=iw.SelectMultiple(options=comp_list,value=[comp_list[0],],rows=10,description='Y-2nd axis:',disabled=False) if "comps" not in Choice.__name__ else iw.SelectMultiple(options=comp_list,value=[comp_list[0],],rows=10,description='Y-2nd axis:',disabled=True)
    Select_series=iw.SelectMultiple(options=series_list,value=series_list,rows=10,description='Series:',disabled=False)
    Select_times=iw.SelectMultiple(options=tuple(df[df.columns[0]]),value=tuple(df[df.columns[0]]),rows=10,description=df.columns[0],disabled=False)
    inter2=iw.interactive(Choice,{'manual' : True, 'manual_name' : 'Plot'},comp_list=Select_comps,comp_list2=Select_comps2,series_list=Select_series,times_list=Select_times)
    controls2 = iw.HBox(inter2.children[:-1])
    out2=inter2.children[-1]
    display(controls2,out2,inter2.manual_button)

def Plot_df(Nr,IDc,r_s=False):
  inter=iw.interactive(DF_build,Nr=iw.IntSlider(value=Nr,min=1,max=5,description="N° Replicates"),
                    IDc=iw.IntSlider(value=IDc,min=0,max=20,description="ID char pos."),
                    Choice=iw.Dropdown(description="Plot Type",
                                       options=[("Series (Bars)",Plotbar_series),
                                                ("Series (Lines)",Plotline_series),
                                                ("Compounds (Bars)",Plotbar_comps),
                                                ("Compounds (Lines)",Plotline_comps),
                                                ("All-in-1 (Lines)",Plotline_all)],
                                        value=None,style={'description_width': 'full'}),
                       replace_series=iw.fixed(r_s),
                       df1=iw.fixed(df))
  controls = iw.HBox(inter.children[:-1])
  out=inter.children[-1]
  display(controls,out)